# [Boosters] Raiffeisen Data Cup. Baseline + собственные фичи
Общий подход:
- Добавляем к каждой транзакции столбец: is_work (если транзакция находится в пределах 0.02 от дома клиента)
- Добавляем к каждой транзакции столбец: is_home (если транзакция находится в пределах 0.02 от работы клиента)
- Обучаем классификатор предсказывающий вероятность (is_home == 1) для транзакции
- Обучаем классификатор предсказывающий вероятность (is_work == 1) для транзакции


**Добавления к baseline**:
- обработка даты дополнена импортом Производственного календаря на соответствующий период (добавлены помимо выходных праздники и предпраздничные дни)
- категориальный признак MCC в результате парсинга по внешнему файлу дал расшифровку кодов торговых точек (два признака: тип и группа)
- расстояние от условного географического центра транзакций клиента (по mean) до каждой транзакции клиента
- бинарный признак ('is_work/home_wide'), означающий принадлежность транзакции к более широкой окрестности, чем в задаче (0.05), использовался на втором этапе обучения (модель на первом этапе обучалась, чтобы получить его, а на втором - используя и его при обучении на трайне) 
- категория клиентов по расходам на еду и товары первой необходимости (MCC - grocery)

Точность определения местоположения:
- для классификатора is_home: ~3x%
- для классификатора is_work: ~2x%
- **оценка на Public Leaderboard: 33.26%, при максимуме на LB 43.95% (21.03.18)**


In [5]:
import pandas as pd
import numpy as np
import datetime

import xgboost as xgb
import sklearn

from sklearn.model_selection import train_test_split
import math
from sklearn.preprocessing import LabelEncoder


In [2]:
# Определим типы колонок для экономии памяти
dtypes = {
    'transaction_date': str,
    'atm_address': str,
    'country': str,
    'city': str,
    'amount': np.float32,
    'currency': np.float32,
    'mcc': str,
    'customer_id': str,
    'pos_address': str,
    'atm_address': str,
    'pos_adress_lat': np.float32,
    'pos_adress_lon': np.float32,
    'pos_address_lat': np.float32,
    'pos_address_lon': np.float32,
    'atm_address_lat': np.float32,
    'atm_address_lon': np.float32,
    'home_add_lat': np.float32,
    'home_add_lon': np.float32,
    'work_add_lat': np.float32,
    'work_add_lon': np.float32,
}

# для экономии памяти будем загружать только часть атрибутов транзакций
usecols_train = ['customer_id','transaction_date','amount','country', 'city', 'currency', 'mcc', 'pos_adress_lat', 'pos_adress_lon', 'atm_address_lat', 'atm_address_lon','home_add_lat','home_add_lon','work_add_lat','work_add_lon']
usecols_test = ['customer_id','transaction_date','amount','country', 'city', 'currency', 'mcc', 'pos_address_lat', 'pos_address_lon', 'atm_address_lat', 'atm_address_lon']

## Читаем train_set, test_set, соединяем в один датасет

In [3]:
train = pd.read_csv('train_set.csv', dtype = dtypes, usecols = usecols_train)
train.rename(columns = {'pos_adress_lat': 'pos_address_lat', 'pos_adress_lon': 'pos_address_lon'}, inplace = True)

test = pd.read_csv('test_set (1).csv', dtype = dtypes, usecols = usecols_test)
submission = pd.DataFrame(test['customer_id'].unique(), columns = ['_ID_'])

# соединяем test/train в одном DataFrame
train['is_train'] = np.int32(1)
test['is_train'] = np.int32(0)
dt = pd.concat([train, test])

del train, test

### Обрабатываем  категориальные признаки

In [4]:
dt['currency'] = dt['currency'].fillna(-1).astype(np.int32)
dt['city'] = dt['city'].factorize()[0].astype(np.int32)
dt['country'] = dt['country'].factorize()[0].astype(np.int32)


# удаляем транзакции без даты
dt.drop(dt[dt['transaction_date'].isnull()].index, axis = 0, inplace = True)
dt['transaction_date'] = dt['transaction_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

# преобразования с датой
dt['transaction_date_dayofweek'] = dt['transaction_date'].dt.dayofweek

dt['transaction_date_day'] = dt['transaction_date'].dt.day
dt['transaction_date_month'] = dt['transaction_date'].dt.month
dt['transaction_date_year'] = dt['transaction_date'].dt.year

### + добавления к baseline (маппинг производственного календаря)

In [5]:
from datetime import datetime

dates = pd.read_csv('calendar.csv', skiprows=1, index_col=0,
                    usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],  
                    names=['year', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])

# предпраздничные дни
pre_holidays = []
# праздничне дни
holidays = []

for year in dates.index:
    for month in dates.columns:
        for day in dates.loc[year, month].split(','):
            if day.endswith('*'):
                pre_holidays.append(datetime(year, int(month), int(day[:len(day)-1]), 0, 0))
            else:
                holidays.append(datetime(year, int(month), int(day), 0, 0))
                
                
# функция, присваивающая лейбл дня
def determ(date, hol, pre_hol):
    if date in hol:
        return 'holiday'
    elif date in pre_hol:
        return 'pre-holiday'
    else: return 'workday'
    

calendar = pd.DataFrame(data={'date': pd.date_range('1/1/2017', '31/12/2017', freq='D')})
calendar['status'] = calendar['date'].apply(lambda x: determ(x, holidays, pre_holidays))

dict_calendar = calendar.set_index('date')['status'].to_dict()
dt['date_type'] = dt['transaction_date'].map(dict_calendar)

### +добавления к baseline - MCC

In [6]:
# обрабатываем мсс по-другому

mcc = pd.read_csv('Spravochnik_MCC_Cod w9l.csv', names= ['mcc', 'type', 'mcc group','title'], sep = ',')
# считывается по-разному - следить, из-за этого бывают ошибки

dict_mcc_group = mcc.set_index('mcc')['mcc group'].to_dict()
dict_mcc_type = mcc.set_index('mcc')['type'].to_dict()

dt['mcc_group'] = dt['mcc'].map(dict_mcc_group).str.lower().str.strip()
dt['mcc_type'] = dt['mcc'].map(dict_mcc_type).str.lower().str.strip()

In [7]:
# LE для типа дня и мсс

le_date_type = LabelEncoder()
le_mcc_group = LabelEncoder()
le_mcc_type = LabelEncoder()

dt['le_date_type'] = le_date_type.fit_transform(dt['date_type'].astype(str))
dt['le_mcc_group'] = le_mcc_group.fit_transform(dt['mcc_group'].astype(str))
dt['le_mcc_type'] = le_mcc_type.fit_transform(dt['mcc_type'].astype(str))


dt.drop(['date_type', 'mcc_group', 'mcc_type'], axis = 1, inplace = True)

### Приводим адрес транзакции для pos и atm-транзакций к единообразному виду

In [8]:
dt['is_atm'] = (~dt['atm_address_lat'].isnull()).astype(np.int32)
dt['is_pos'] = (~dt['pos_address_lat'].isnull()).astype(np.int32)

dt['address_lat'] = dt['atm_address_lat'].fillna(0) + dt['pos_address_lat'].fillna(0)
dt['address_lon'] = dt['atm_address_lon'].fillna(0) + dt['pos_address_lon'].fillna(0)

dt.drop(['atm_address_lat','atm_address_lon','pos_address_lat','pos_address_lon'], axis = 1, inplace = True)

# удалим транзакции без адреса
dt.drop(dt[((dt['address_lon'] == 0) & (dt['address_lon'] == 0))].index, axis = 0, inplace = True)

### Генерируем признаки is_home, is_work

In [9]:
lat = dt['home_add_lat'] - dt['address_lat']
lon = dt['home_add_lon'] - dt['address_lon']
dt['is_home'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int32)
dt['has_home'] = (~dt['home_add_lon'].isnull()).astype(np.int32)
dt['is_home_wide'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.05).astype(np.int32) # уточнение окрестности
dt['has_home_wide'] = (~dt['home_add_lon'].isnull()).astype(np.int32)


lat = dt['work_add_lat'] - dt['address_lat']
lon = dt['work_add_lon'] - dt['address_lon']
dt['is_work'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int32)
dt['has_work'] = (~dt['work_add_lon'].isnull()).astype(np.int32)
dt['is_work_wide'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.05).astype(np.int32)
dt['has_work_wide'] = (~dt['work_add_lon'].isnull()).astype(np.int32)


dt.drop(['work_add_lat','work_add_lon','home_add_lat','home_add_lon'], axis = 1, inplace = True)

### Генерируем категориальный признак для адреса

In [10]:
dt['address'] = dt['address_lat'].apply(lambda x: "%.02f" % x) + ';' + dt['address_lon'].apply(lambda x: "%.02f" % x)
dt['address'] = dt['address'].factorize()[0].astype(np.int32)

### Генерируем несколько абонентских фич

In [11]:
# количество транзакций каждого клиента
dt = dt.merge(dt.groupby('customer_id')['amount'].count().reset_index(name = 'tx'), how = 'left')
dt['tx'] = dt['tx'].astype(np.int32)

dt = dt.merge(dt.groupby(['customer_id','address'])['amount'].count().reset_index(name = 'tx_cust_addr'), how = 'left')
dt['tx_cust_addr'] = dt['tx_cust_addr'].astype(np.int32)

# какая часть транзакций клиента приходится на данный адрес
dt['ratio1'] = dt['tx_cust_addr'] / dt['tx']

### +добавления к baseline - расстояния до географического центра транзакций

In [12]:
dict_mean_for_trans_lat = dt.groupby(['customer_id'])['address_lat'].mean().to_dict()
dict_mean_for_trans_lon = dt.groupby(['customer_id'])['address_lon'].mean().to_dict()

dt['mean_for_trans_lat'] = dt['customer_id'].map(dict_mean_for_trans_lat)
dt['mean_for_trans_lon'] = dt['customer_id'].map(dict_mean_for_trans_lon)

In [13]:
def formula_dist(lat_h, lon_h, lat_c, lon_c):
    rad = 6372795
    
    lat1 = lat_h*math.pi/180.
    lat2 = lat_c*math.pi/180.
    long1 = lon_h*math.pi/180.
    long2 = lon_c*math.pi/180.
    
    cl1 = math.cos(lat1)
    cl2 = math.cos(lat2)
    sl1 = math.sin(lat1)
    sl2 = math.sin(lat2)
    delta = long2 - long1
    cdelta = math.cos(delta)
    sdelta = math.sin(delta)
    
    y = math.sqrt(math.pow(cl2*sdelta,2)+math.pow(cl1*sl2-sl1*cl2*cdelta,2))
    x = sl1*sl2+cl1*cl2*cdelta
    ad = math.atan2(y,x)
    dist = ad*rad
    
    return dist

In [14]:
%%time
dt['dist_terminal_to_mean_terminal'] = dt.apply(lambda row: formula_dist(row['address_lat'], row['address_lon'], row['mean_for_trans_lat'], row['mean_for_trans_lon']), axis=1)


CPU times: user 2min 28s, sys: 641 ms, total: 2min 28s
Wall time: 2min 28s


## по расходам не сделано

In [26]:
quantile_list = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99, 1.]


quantile_labels = ['Q_0-10', 'Q_10-20', 'Q_20-30', 'Q_30-40', 'Q_40-50', 'Q_50-60', 'Q_60-70', 'Q_70-80',
                    'Q_80-90', 'Q_90-95', 'Q_95-99', 'Q_99-100']
                   
dt['Spend_groc_quantile_range'] = pd.qcut(
                                           dt['amount_rub_643_978_840_975_203_980_933'],
                                           q=quantile_list)
dt['Spending_groc_quantile_label'] = pd.qcut(
                                           dt['amount_rub_643_978_840_975_203_980_933'],
                                           q=quantile_list,      
                                           labels=quantile_labels)

KeyError: 'amount_rub_643_978_840_975_203_980_933'

In [33]:
dt.drop('amount_init', axis =1,  inplace = True)

## Вспомогательные функции для оценки точности классификатора

In [15]:
def _best(x):
    ret = None
    for col in ys:
        pred = ('pred:%s' % col)
        if pred in x:
            i = (x[pred].idxmax())
            cols = [pred,'address_lat','address_lon']
            if col in x:
                cols.append(col)
            tmp = x.loc[i,cols]
            tmp.rename({
                'address_lat':'%s:add_lat' % col,
                'address_lon':'%s:add_lon' % col,
            }, inplace = True)
            if ret is None:
                ret = tmp
            else:
                ret = pd.concat([ret, tmp])
    return ret

In [37]:
def predict_proba(dt, ys = ['is_home_wide', 'is_work_wide']):
    for col in ys:
        pred = ('pred:%s' % col)
        dt[pred] = model[col].predict_proba(dt[xs])[:,1]
    return dt.groupby('customer_id').apply(_best).reset_index()

In [53]:
def score(dt, ys = ['is_home_wide', 'is_work_wide']):
    dt_ret = predict_proba(dt, ys)
    mean = 0.0
    for col in ys:
        col_mean = dt_ret[col].mean()
        mean += col_mean
    if len(ys) == 2:
        mean = mean / len(ys)
    return mean

# Первый этап обучения
### добавления к baseline - обучение на метку более широкой окрестности

### Признаки, на которых будем обучать модель

In [36]:
xs = ['amount','currency','city','country', 'is_atm','is_pos','ratio1', 
      'transaction_date_dayofweek', 'transaction_date_day', 
      'transaction_date_month', 'transaction_date_year', 
      'dist_terminal_to_mean_terminal', 'address',
      'le_date_type', 'le_mcc_group', 'le_mcc_type']
ys = ['is_home_wide', 'is_work_wide']

# Создаем классификаторы
**Hint**: можно поигратьcя с гиперпараметрами для лучшего результата :)

In [34]:
model0 = {
    'is_home_wide': xgb.XGBClassifier(n_estimators = 100, n_jobs = 3),
    'is_work_wide': xgb.XGBClassifier(n_estimators = 100, n_jobs = 3),
}

# Обучаем классификаторы

In [38]:
model = {}

# последовательно обучаем два классификатора
for col in ['is_home_wide', 'is_work_wide']:
    
    #выберем для обучение транзакции только тех клиентов из train, у которых хоть в одной транзакции указано место работы/жительства
    cust_train = dt[dt['is_train'] == 1].groupby('customer_id')[col.replace('is_','has_')].max()
    cust_train = cust_train[cust_train > 0].index
    
    #разобъем train на train/valid для валидации
    cust_train, cust_valid = train_test_split(cust_train, test_size = 0.1, shuffle = True, random_state = 2)
    
    train = pd.DataFrame(cust_train, columns = ['customer_id']).merge(dt, how = 'left')
    valid = pd.DataFrame(cust_valid, columns = ['customer_id']).merge(dt, how = 'left')

    print ("Training:", col)
    clf = sklearn.base.clone(model0[col])
    clf.fit(train[xs], train[col], eval_metric = 'logloss', eval_set = [(train[xs], train[col]), (valid[xs], valid[col])], verbose=10)
    model[col] = clf
    print ("Train accuracy:", score(train, ys = [col]))
    print ("Test accuracy:", score(valid, ys = [col]))
    print ()


Training: is_home_wide
[0]	validation_0-logloss:0.679965	validation_1-logloss:0.679461
[10]	validation_0-logloss:0.618609	validation_1-logloss:0.614631
[20]	validation_0-logloss:0.600859	validation_1-logloss:0.597187
[30]	validation_0-logloss:0.592416	validation_1-logloss:0.590149
[40]	validation_0-logloss:0.586453	validation_1-logloss:0.584629
[50]	validation_0-logloss:0.582619	validation_1-logloss:0.581431
[60]	validation_0-logloss:0.580247	validation_1-logloss:0.579961
[70]	validation_0-logloss:0.577943	validation_1-logloss:0.578647
[80]	validation_0-logloss:0.576109	validation_1-logloss:0.57707
[90]	validation_0-logloss:0.574595	validation_1-logloss:0.575845
[99]	validation_0-logloss:0.573332	validation_1-logloss:0.575266
Train accuracy: 0.5216666666666666
Test accuracy: 0.521

Training: is_work_wide
[0]	validation_0-logloss:0.67203	validation_1-logloss:0.671854
[10]	validation_0-logloss:0.579685	validation_1-logloss:0.580185
[20]	validation_0-logloss:0.556008	validation_1-logloss:

**Признак попадания точки в шировкую окрестность записываем как  новую фичу, сохраняем файл и потом проводим повторное обучение**

In [42]:
for col in ['is_home_wide', 'is_work_wide']:
    pred = ('pred:%s' % col)
    dt[pred] = model[col].predict_proba(dt[xs])[:,1]
    print (dt[pred].head())

0    0.758286
1    0.670079
2    0.672531
3    0.758286
4    0.670079
Name: pred:is_home_wide, dtype: float32
0    0.205382
1    0.255098
2    0.282859
3    0.212399
4    0.247112
Name: pred:is_work_wide, dtype: float32


In [44]:
dt.columns

Index(['amount', 'city', 'country', 'currency', 'customer_id', 'is_train',
       'mcc', 'transaction_date', 'transaction_date_dayofweek',
       'transaction_date_day', 'transaction_date_month',
       'transaction_date_year', 'le_date_type', 'le_mcc_group', 'le_mcc_type',
       'is_atm', 'is_pos', 'address_lat', 'address_lon', 'is_home', 'has_home',
       'is_home_wide', 'has_home_wide', 'is_work', 'has_work', 'is_work_wide',
       'has_work_wide', 'address', 'tx', 'tx_cust_addr', 'ratio1',
       'mean_for_trans_lat', 'mean_for_trans_lon',
       'dist_terminal_to_mean_terminal', 'pred:is_home_wide',
       'pred:is_work_wide'],
      dtype='object')

In [73]:
dt_col = dt.columns
dt.to_csv('dt_backup_s6_1.csv', sep = ',', columns=dt_col, header=True, index = False)


In [74]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2172472 entries, 0 to 2172471
Data columns (total 36 columns):
amount                            float32
city                              int32
country                           int32
currency                          int32
customer_id                       object
is_train                          int32
mcc                               object
transaction_date                  datetime64[ns]
transaction_date_dayofweek        int64
transaction_date_day              int64
transaction_date_month            int64
transaction_date_year             int64
le_date_type                      int64
le_mcc_group                      int64
le_mcc_type                       int64
is_atm                            int32
is_pos                            int32
address_lat                       float32
address_lon                       float32
is_home                           int32
has_home                          int32
is_home_wide                  

# Второй этап обучения
Тетрадка перезапускалась

In [3]:
dt = pd.read_csv('dt_backup_s6_1.csv', low_memory=False)

In [3]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2172472 entries, 0 to 2172471
Data columns (total 36 columns):
amount                            float64
city                              int64
country                           int64
currency                          int64
customer_id                       object
is_train                          int64
mcc                               object
transaction_date                  object
transaction_date_dayofweek        int64
transaction_date_day              int64
transaction_date_month            int64
transaction_date_year             int64
le_date_type                      int64
le_mcc_group                      int64
le_mcc_type                       int64
is_atm                            int64
is_pos                            int64
address_lat                       float64
address_lon                       float64
is_home                           int64
has_home                          int64
is_home_wide                      int6

### + добавления к baseline - категория клиента по расходам на еду (adaptive bins)

In [6]:
# возвращаем прологарифмированные значения признака расходов
dt['amount_init'] = np.exp(dt['amount'])*100

# вводим функцию перевода из иностранных валют. 
# Для наиболее частых - по каждому месяцу, для менее частых - по среднему за год. 
# Самые низкочастотные получают значение -9999 (для xgboost это допустимо)

def curr(amount, currency, month):
    if (currency == 978) & (month == 1):
        return amount * 63.56
    if (currency == 978) & (month == 2):
        return amount * 62.40
    if (currency == 978) & (month == 3):
        return amount * 62.00
    if (currency == 978) & (month == 4):
        return amount * 60.46
    if (currency == 978) & (month == 5):
        return amount * 62.95
    if (currency == 978) & (month == 6):
        return amount * 64.96
    if (currency == 978) & (month == 7):
        return amount * 68.62
    if (currency == 978) & (month == 8):
        return amount * 70.37
    if (currency == 978) & (month == 9):
        return amount * 68.79
    if (currency == 978) & (month == 10):
        return amount * 67.85
    if (currency == 978) & (month == 11):
        return amount * 69.13
    if (currency == 978) & (month == 12):
        return amount * 69.33
    
    if (currency == 840) & (month == 1):
        return amount * 59.62
    if (currency == 840) & (month == 2):
        return amount * 58.53
    if (currency == 840) & (month == 3):
        return amount * 58.00
    if (currency == 840) & (month == 4):
        return amount * 56.43
    if (currency == 840) & (month == 5):
        return amount * 56.95
    if (currency == 840) & (month == 6):
        return amount * 57.89
    if (currency == 840) & (month == 7):
        return amount * 59.69
    if (currency == 840) & (month == 8):
        return amount * 59.61
    if (currency == 840) & (month == 9):
        return amount * 57.74
    if (currency == 840) & (month == 10):
        return amount * 57.69
    if (currency == 840) & (month == 11):
        r6eturn amount * 58.92
    if (currency == 840) & (month == 12):
        return amount * 58.57
    
    if currency == 643:
        return amount 
    
    if currency == 975:
        return amount * 33.69
    
    if currency == 203:
        return amount * 2.5
    
    if currency == 980:
        return amount * 2.19
    
    if currency == 933:
        return amount * 30.21
    
    else:
        return -9999

In [8]:
dt['amount_rub'] = dt.apply(lambda row: curr(row['amount_init'], row['currency'], row['transaction_date_month']), axis=1)

dt.drop(['amount', 'amount_init'], axis = 1, inplace = True)

In [11]:
# adapred binning
quantile_list = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99, 1.]

quantile_labels = ['Q_0-10', 'Q_10-20', 'Q_20-30', 'Q_30-40', 'Q_40-50', 'Q_50-60', 'Q_60-70', 'Q_70-80',
                    'Q_80-90', 'Q_90-95', 'Q_95-99', 'Q_99-100']
                   
dt['Spend_groc_quantile_range'] = pd.qcut(
                                           dt['amount_rub'],
                                           q=quantile_list)
dt['Spending_groc_quantile_label'] = pd.qcut(
                                           dt['amount_rub'],
                                           q=quantile_list,      
                                           labels=quantile_labels)

In [18]:
le_spending_groc = LabelEncoder()

dt['Spending_groc_label'] = le_spending_groc.fit_transform(dt['Spending_groc_quantile_label'].astype(str))

In [19]:
xs = ['amount_rub','currency','city','country', 'is_atm','is_pos','ratio1', 
      'transaction_date_dayofweek', 'transaction_date_day', 
      'transaction_date_month', 'transaction_date_year', 
      'dist_terminal_to_mean_terminal', 'address',
      'le_date_type', 'le_mcc_group', 'le_mcc_type', 
      'pred:is_home_wide', 'pred:is_work_wide', 'Spending_groc_label']
ys = ['is_home', 'is_work']

In [13]:
def _best(x):
    ret = None
    for col in ys:
        pred = ('pred:%s' % col)
        if pred in x:
            i = (x[pred].idxmax())
            cols = [pred,'address_lat','address_lon']
            if col in x:
                cols.append(col)
            tmp = x.loc[i,cols]
            tmp.rename({
                'address_lat':'%s:add_lat' % col,
                'address_lon':'%s:add_lon' % col,
            }, inplace = True)
            if ret is None:
                ret = tmp
            else:
                ret = pd.concat([ret, tmp])
    return ret

In [14]:
def predict_proba(dt, ys = ['is_home', 'is_work']):
    for col in ys:
        pred = ('pred:%s' % col)
        dt[pred] = model[col].predict_proba(dt[xs])[:,1]
    return dt.groupby('customer_id').apply(_best).reset_index()

In [15]:
def score(dt, ys = ['is_home', 'is_work']):
    dt_ret = predict_proba(dt, ys)
    mean = 0.0
    for col in ys:
        col_mean = dt_ret[col].mean()
        mean += col_mean
    if len(ys) == 2:
        mean = mean / len(ys)
    return mean

In [16]:
model0 = {
    'is_home': xgb.XGBClassifier(n_estimators = 100, n_jobs = 3),
    'is_work': xgb.XGBClassifier(n_estimators = 100, n_jobs = 3),
}

In [20]:
%%time

model = {}

# последовательно обучаем два классификатора
for col in ['is_home', 'is_work']:
    
    #выберем для обучение транзакции только тех клиентов из train, у которых хоть в одной транзакции указано место работы/жительства
    cust_train = dt[dt['is_train'] == 1].groupby('customer_id')[col.replace('is_','has_')].max()
    cust_train = cust_train[cust_train > 0].index
    
    #разобъем train на train/valid для валидации
    cust_train, cust_valid = train_test_split(cust_train, test_size = 0.1, shuffle = True, random_state = 2)
    
    train = pd.DataFrame(cust_train, columns = ['customer_id']).merge(dt, how = 'left')
    valid = pd.DataFrame(cust_valid, columns = ['customer_id']).merge(dt, how = 'left')

    print ("Training:", col)
    clf = sklearn.base.clone(model0[col])
    clf.fit(train[xs], train[col], eval_metric = 'logloss', eval_set = [(train[xs], train[col]), (valid[xs], valid[col])], verbose=10)
    model[col] = clf
    print ("Train accuracy:", score(train, ys = [col]))
    print ("Test accuracy:", score(valid, ys = [col]))
    print ()


Training: is_home
[0]	validation_0-logloss:0.65756	validation_1-logloss:0.657045
[10]	validation_0-logloss:0.509359	validation_1-logloss:0.507298
[20]	validation_0-logloss:0.479724	validation_1-logloss:0.478529
[30]	validation_0-logloss:0.472407	validation_1-logloss:0.471876
[40]	validation_0-logloss:0.469401	validation_1-logloss:0.469608
[50]	validation_0-logloss:0.467466	validation_1-logloss:0.468901
[60]	validation_0-logloss:0.465726	validation_1-logloss:0.468308
[70]	validation_0-logloss:0.464307	validation_1-logloss:0.467815
[80]	validation_0-logloss:0.462836	validation_1-logloss:0.467648
[90]	validation_0-logloss:0.461544	validation_1-logloss:0.467266
[99]	validation_0-logloss:0.460039	validation_1-logloss:0.466659
Train accuracy: 0.3894444444444444
Test accuracy: 0.378

Training: is_work
[0]	validation_0-logloss:0.643408	validation_1-logloss:0.644452
[10]	validation_0-logloss:0.433208	validation_1-logloss:0.4432
[20]	validation_0-logloss:0.388805	validation_1-logloss:0.404559
[3

### для сравнения: результаты без фичи категории по расходам на еду:

**Training: is_home**<div>
Train accuracy: 0.3903333333333333

Test accuracy: 0.379 

**Training: is_work**<div>
Train accuracy: 0.27789745799224475

Test accuracy: 0.2558139534883721


#### Результат по лидерборду: 33.1%

### поиск по сетке параметров

In [26]:
from sklearn.model_selection import GridSearchCV 

In [30]:
%%time

model = {}
params = {
    'n_estimators': [100, 150, 200],
    'max_depth': [3, 4, 5, 6]
}

# последовательно обучаем два классификатора
for col in ['is_home', 'is_work']:
    
    #выберем для обучение транзакции только тех клиентов из train, у которых хоть в одной транзакции указано место работы/жительства
    cust_train = dt[dt['is_train'] == 1].groupby('customer_id')[col.replace('is_','has_')].max()
    cust_train = cust_train[cust_train > 0].index
    
    #разобъем train на train/valid для валидации
    cust_train, cust_valid = train_test_split(cust_train, test_size = 0.1, shuffle = True, random_state = 2)
    
    train = pd.DataFrame(cust_train, columns = ['customer_id']).merge(dt, how = 'left')
    valid = pd.DataFrame(cust_valid, columns = ['customer_id']).merge(dt, how = 'left')

    print ("Training:", col)
    clf = sklearn.base.clone(model0[col])
    clf_grid_search = GridSearchCV(clf, params, cv=3, n_jobs=1)
    clf_grid_search.fit(train[xs], train[col], eval_metric = 'logloss', eval_set = [(train[xs], train[col]), (valid[xs], valid[col])], verbose=10)
    model[col] = clf_grid_search.best_estimator_
    print ("Train accuracy:", score(train, ys = [col]))
    print ("Test accuracy:", score(valid, ys = [col]))
    print ()

Training: is_home
[0]	validation_0-logloss:0.657641	validation_1-logloss:0.657164
[10]	validation_0-logloss:0.509509	validation_1-logloss:0.507247
[20]	validation_0-logloss:0.479939	validation_1-logloss:0.478405
[30]	validation_0-logloss:0.472662	validation_1-logloss:0.471782
[40]	validation_0-logloss:0.469869	validation_1-logloss:0.46964
[50]	validation_0-logloss:0.467972	validation_1-logloss:0.468732
[60]	validation_0-logloss:0.466435	validation_1-logloss:0.46862
[70]	validation_0-logloss:0.465014	validation_1-logloss:0.468291
[80]	validation_0-logloss:0.463809	validation_1-logloss:0.468331
[90]	validation_0-logloss:0.462529	validation_1-logloss:0.467837
[99]	validation_0-logloss:0.461588	validation_1-logloss:0.467688
[0]	validation_0-logloss:0.657546	validation_1-logloss:0.657184
[10]	validation_0-logloss:0.509238	validation_1-logloss:0.507232
[20]	validation_0-logloss:0.479742	validation_1-logloss:0.478444
[30]	validation_0-logloss:0.472571	validation_1-logloss:0.472195
[40]	valida

[30]	validation_0-logloss:0.472295	validation_1-logloss:0.471738
[40]	validation_0-logloss:0.469498	validation_1-logloss:0.469726
[50]	validation_0-logloss:0.467671	validation_1-logloss:0.468983
[60]	validation_0-logloss:0.466083	validation_1-logloss:0.469154
[70]	validation_0-logloss:0.464806	validation_1-logloss:0.469008
[80]	validation_0-logloss:0.463564	validation_1-logloss:0.468545
[90]	validation_0-logloss:0.462117	validation_1-logloss:0.46864
[100]	validation_0-logloss:0.460986	validation_1-logloss:0.468716
[110]	validation_0-logloss:0.460036	validation_1-logloss:0.468867
[120]	validation_0-logloss:0.458937	validation_1-logloss:0.468623
[130]	validation_0-logloss:0.457958	validation_1-logloss:0.468651
[140]	validation_0-logloss:0.456934	validation_1-logloss:0.469341
[150]	validation_0-logloss:0.456239	validation_1-logloss:0.469679
[160]	validation_0-logloss:0.455503	validation_1-logloss:0.469887
[170]	validation_0-logloss:0.454853	validation_1-logloss:0.470399
[180]	validation_0

[70]	validation_0-logloss:0.458255	validation_1-logloss:0.466538
[80]	validation_0-logloss:0.456366	validation_1-logloss:0.466626
[90]	validation_0-logloss:0.454115	validation_1-logloss:0.466763
[100]	validation_0-logloss:0.452301	validation_1-logloss:0.466402
[110]	validation_0-logloss:0.451187	validation_1-logloss:0.467024
[120]	validation_0-logloss:0.4497	validation_1-logloss:0.468017
[130]	validation_0-logloss:0.448659	validation_1-logloss:0.467947
[140]	validation_0-logloss:0.446817	validation_1-logloss:0.467772
[150]	validation_0-logloss:0.445554	validation_1-logloss:0.46802
[160]	validation_0-logloss:0.444028	validation_1-logloss:0.468547
[170]	validation_0-logloss:0.443027	validation_1-logloss:0.469052
[180]	validation_0-logloss:0.442113	validation_1-logloss:0.469869
[190]	validation_0-logloss:0.441045	validation_1-logloss:0.470569
[199]	validation_0-logloss:0.439945	validation_1-logloss:0.470263
[0]	validation_0-logloss:0.657174	validation_1-logloss:0.656953
[10]	validation_0-

[100]	validation_0-logloss:0.443111	validation_1-logloss:0.466295
[110]	validation_0-logloss:0.440346	validation_1-logloss:0.466313
[120]	validation_0-logloss:0.43853	validation_1-logloss:0.466548
[130]	validation_0-logloss:0.436362	validation_1-logloss:0.46668
[140]	validation_0-logloss:0.434053	validation_1-logloss:0.466572
[150]	validation_0-logloss:0.431735	validation_1-logloss:0.466531
[160]	validation_0-logloss:0.429804	validation_1-logloss:0.466729
[170]	validation_0-logloss:0.428464	validation_1-logloss:0.467026
[180]	validation_0-logloss:0.426636	validation_1-logloss:0.467715
[190]	validation_0-logloss:0.425102	validation_1-logloss:0.467857
[199]	validation_0-logloss:0.423929	validation_1-logloss:0.467773
[0]	validation_0-logloss:0.656736	validation_1-logloss:0.656669
[10]	validation_0-logloss:0.504306	validation_1-logloss:0.504358
[20]	validation_0-logloss:0.472534	validation_1-logloss:0.475135
[30]	validation_0-logloss:0.464163	validation_1-logloss:0.468552
[40]	validation_0

[80]	validation_0-logloss:0.437661	validation_1-logloss:0.466344
[90]	validation_0-logloss:0.434409	validation_1-logloss:0.466244
[100]	validation_0-logloss:0.43106	validation_1-logloss:0.465348
[110]	validation_0-logloss:0.427834	validation_1-logloss:0.464922
[120]	validation_0-logloss:0.425158	validation_1-logloss:0.464752
[130]	validation_0-logloss:0.422288	validation_1-logloss:0.465317
[140]	validation_0-logloss:0.420152	validation_1-logloss:0.46501
[149]	validation_0-logloss:0.418179	validation_1-logloss:0.46446
[0]	validation_0-logloss:0.656476	validation_1-logloss:0.656271
[10]	validation_0-logloss:0.501771	validation_1-logloss:0.50383
[20]	validation_0-logloss:0.467983	validation_1-logloss:0.473947
[30]	validation_0-logloss:0.458278	validation_1-logloss:0.46765
[40]	validation_0-logloss:0.453534	validation_1-logloss:0.467414
[50]	validation_0-logloss:0.449036	validation_1-logloss:0.466582
[60]	validation_0-logloss:0.445385	validation_1-logloss:0.466545
[70]	validation_0-logloss

[100]	validation_0-logloss:0.361993	validation_1-logloss:0.398256
[110]	validation_0-logloss:0.36076	validation_1-logloss:0.398774
[120]	validation_0-logloss:0.359388	validation_1-logloss:0.399663
[130]	validation_0-logloss:0.357448	validation_1-logloss:0.400097
[140]	validation_0-logloss:0.355857	validation_1-logloss:0.400734
[149]	validation_0-logloss:0.354874	validation_1-logloss:0.400945
[0]	validation_0-logloss:0.643371	validation_1-logloss:0.644158
[10]	validation_0-logloss:0.433654	validation_1-logloss:0.444072
[20]	validation_0-logloss:0.389635	validation_1-logloss:0.405529
[30]	validation_0-logloss:0.378744	validation_1-logloss:0.397906
[40]	validation_0-logloss:0.374403	validation_1-logloss:0.396692
[50]	validation_0-logloss:0.371737	validation_1-logloss:0.395185
[60]	validation_0-logloss:0.369486	validation_1-logloss:0.395359
[70]	validation_0-logloss:0.367481	validation_1-logloss:0.395637
[80]	validation_0-logloss:0.365691	validation_1-logloss:0.396001
[90]	validation_0-log

[30]	validation_0-logloss:0.371666	validation_1-logloss:0.398885
[40]	validation_0-logloss:0.366589	validation_1-logloss:0.396916
[50]	validation_0-logloss:0.362958	validation_1-logloss:0.396207
[60]	validation_0-logloss:0.360489	validation_1-logloss:0.396381
[70]	validation_0-logloss:0.35814	validation_1-logloss:0.396572
[80]	validation_0-logloss:0.355535	validation_1-logloss:0.398189
[90]	validation_0-logloss:0.353108	validation_1-logloss:0.398328
[99]	validation_0-logloss:0.350783	validation_1-logloss:0.397319
[0]	validation_0-logloss:0.642659	validation_1-logloss:0.644085
[10]	validation_0-logloss:0.429447	validation_1-logloss:0.441773
[20]	validation_0-logloss:0.384165	validation_1-logloss:0.403517
[30]	validation_0-logloss:0.372512	validation_1-logloss:0.39698
[40]	validation_0-logloss:0.367803	validation_1-logloss:0.396251
[50]	validation_0-logloss:0.36371	validation_1-logloss:0.396392
[60]	validation_0-logloss:0.360325	validation_1-logloss:0.395131
[70]	validation_0-logloss:0.3

[70]	validation_0-logloss:0.34643	validation_1-logloss:0.399193
[80]	validation_0-logloss:0.341988	validation_1-logloss:0.39876
[90]	validation_0-logloss:0.338777	validation_1-logloss:0.399801
[99]	validation_0-logloss:0.336079	validation_1-logloss:0.400659
[0]	validation_0-logloss:0.642351	validation_1-logloss:0.64372
[10]	validation_0-logloss:0.426944	validation_1-logloss:0.442474
[20]	validation_0-logloss:0.37958	validation_1-logloss:0.404997
[30]	validation_0-logloss:0.366257	validation_1-logloss:0.396413
[40]	validation_0-logloss:0.359735	validation_1-logloss:0.395032
[50]	validation_0-logloss:0.355156	validation_1-logloss:0.395153
[60]	validation_0-logloss:0.351391	validation_1-logloss:0.393994
[70]	validation_0-logloss:0.349116	validation_1-logloss:0.394197
[80]	validation_0-logloss:0.345478	validation_1-logloss:0.394767
[90]	validation_0-logloss:0.341778	validation_1-logloss:0.393803
[99]	validation_0-logloss:0.339019	validation_1-logloss:0.394288
[0]	validation_0-logloss:0.642

[110]	validation_0-logloss:0.336793	validation_1-logloss:0.399636
[120]	validation_0-logloss:0.333386	validation_1-logloss:0.401656
[130]	validation_0-logloss:0.330016	validation_1-logloss:0.401991
[140]	validation_0-logloss:0.327973	validation_1-logloss:0.402425
[150]	validation_0-logloss:0.325266	validation_1-logloss:0.403964
[160]	validation_0-logloss:0.324054	validation_1-logloss:0.405193
[170]	validation_0-logloss:0.321548	validation_1-logloss:0.405108
[180]	validation_0-logloss:0.319785	validation_1-logloss:0.40625
[190]	validation_0-logloss:0.317962	validation_1-logloss:0.407638
[199]	validation_0-logloss:0.316708	validation_1-logloss:0.407992
[0]	validation_0-logloss:0.64158	validation_1-logloss:0.644364
[10]	validation_0-logloss:0.420144	validation_1-logloss:0.441704
[20]	validation_0-logloss:0.370268	validation_1-logloss:0.405984
[30]	validation_0-logloss:0.35554	validation_1-logloss:0.40065
[40]	validation_0-logloss:0.349105	validation_1-logloss:0.399898
[50]	validation_0-lo

[150]	validation_0-logloss:0.305629	validation_1-logloss:0.39237
[160]	validation_0-logloss:0.302894	validation_1-logloss:0.393401
[170]	validation_0-logloss:0.299734	validation_1-logloss:0.392482
[180]	validation_0-logloss:0.296947	validation_1-logloss:0.392653
[190]	validation_0-logloss:0.29583	validation_1-logloss:0.39256
[199]	validation_0-logloss:0.293161	validation_1-logloss:0.39233
[0]	validation_0-logloss:0.641853	validation_1-logloss:0.645525
[10]	validation_0-logloss:0.420499	validation_1-logloss:0.449597
[20]	validation_0-logloss:0.370167	validation_1-logloss:0.412267
[30]	validation_0-logloss:0.354877	validation_1-logloss:0.40379
[40]	validation_0-logloss:0.348179	validation_1-logloss:0.402384
[50]	validation_0-logloss:0.343794	validation_1-logloss:0.402995
[60]	validation_0-logloss:0.339049	validation_1-logloss:0.402733
[70]	validation_0-logloss:0.335669	validation_1-logloss:0.402232
[80]	validation_0-logloss:0.33215	validation_1-logloss:0.403655
[90]	validation_0-logloss:

# Predict

In [31]:
cust_test = dt[dt['is_train'] == 0]['customer_id'].unique()
test = pd.DataFrame(cust_test, columns = ['customer_id']).merge(dt, how = 'left')
test = predict_proba(test)
test.rename(columns = {
        'customer_id':'_ID_',
        'is_home:add_lat': '_HOME_LAT_',
        'is_home:add_lon': '_HOME_LON_',
        'is_work:add_lat': '_WORK_LAT_',
        'is_work:add_lon': '_WORK_LON_'}, inplace = True)
test = test[['_ID_', '_WORK_LAT_', '_WORK_LON_', '_HOME_LAT_', '_HOME_LON_']]

# Формируем submission-файл

In [32]:
# Заполняем пропуски
submission = submission.merge(test, how = 'left').fillna(0)

# Пишем файл submission
submission.to_csv('submit8_on_baseline2.csv', index = False)

## Дальнейшая генерация фич:
- вычисление количества транзакций в окрестности
- близость тразакции к определённым типам объетов (необходимо подтягивать информацию из API 2GIS или Yandex)